## Importing Packages

In [1]:
import pandas as pd

In [2]:
# link to import data
data_url="https://www.berlin.de/sen/uvk/_assets/verkehr/infrastruktur/oeffentliche-toiletten/berliner-toiletten-standorte.xlsx"

## Importing data from source


In [3]:
data = pd.read_excel(data_url)

## Viewing data

In [6]:
data.head()

,Stand:27.04.2022,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LavatoryID,Description,City,Street,Number,PostalCode,Country,Longitude,Latitude,isOwnedByWall,isHandicappedAccessible,Price,canBePayedWithCoins,canBePayedInApp,canBePayedWithNFC,hasChangingTable,LabelID,hasUrinal,FID
3,Wall_101003,Toilette Ottmachauer Steig (oberhalb Badestell...,Berlin,"Krumme Lanke, Quermatenweg (0-24 Uhr)",NaN,14109,Deutschland,"13,240574813","52,453581173",1,0,"0,00",0,0,0,0,5,0,269
4,Wall_112911,Toilette Uferweg,Berlin,"Schlachtensee , Am Schlachtensee ggü. 145 (0-...",NaN,14129,Deutschland,"13,1992287644287","52,4365068880875",1,0,"0,00",0,0,0,0,5,0,NaN


## Fixing data structure

In [9]:
# Making row 2 the column headings
data.columns = data.iloc[2]

In [10]:
data.head()

2,LavatoryID,Description,City,Street,Number,PostalCode,Country,Longitude,Latitude,isOwnedByWall,isHandicappedAccessible,Price,canBePayedWithCoins,canBePayedInApp,canBePayedWithNFC,hasChangingTable,LabelID,hasUrinal,FID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LavatoryID,Description,City,Street,Number,PostalCode,Country,Longitude,Latitude,isOwnedByWall,isHandicappedAccessible,Price,canBePayedWithCoins,canBePayedInApp,canBePayedWithNFC,hasChangingTable,LabelID,hasUrinal,FID
3,Wall_101003,Toilette Ottmachauer Steig (oberhalb Badestell...,Berlin,"Krumme Lanke, Quermatenweg (0-24 Uhr)",NaN,14109,Deutschland,"13,240574813","52,453581173",1,0,"0,00",0,0,0,0,5,0,269
4,Wall_112911,Toilette Uferweg,Berlin,"Schlachtensee , Am Schlachtensee ggü. 145 (0-...",NaN,14129,Deutschland,"13,1992287644287","52,4365068880875",1,0,"0,00",0,0,0,0,5,0,NaN


In [11]:
# dropping redundant rows
data.drop([0,1,2])

2,LavatoryID,Description,City,Street,Number,PostalCode,Country,Longitude,Latitude,isOwnedByWall,isHandicappedAccessible,Price,canBePayedWithCoins,canBePayedInApp,canBePayedWithNFC,hasChangingTable,LabelID,hasUrinal,FID
3,Wall_101003,Toilette Ottmachauer Steig (oberhalb Badestell...,Berlin,"Krumme Lanke, Quermatenweg (0-24 Uhr)",NaN,14109,Deutschland,"13,240574813","52,453581173",1,0,"0,00",0,0,0,0,5,0,269
4,Wall_112911,Toilette Uferweg,Berlin,"Schlachtensee , Am Schlachtensee ggü. 145 (0-...",NaN,14129,Deutschland,"13,1992287644287","52,4365068880875",1,0,"0,00",0,0,0,0,5,0,NaN
5,Wall_115753,Toilette Am Kiesteich 50,Berlin,Spektepark (0-24 Uhr),NaN,13589,Deutschland,"13,1648593254089","52,5488398803196",1,0,"0,00",0,0,0,0,5,0,NaN
6,Wall_116738,"Wall CT, Hubertusdamm",Berlin,Hubertusdamm ggü. 7 (0-24 Uhr),NaN,13125,Deutschland,"13,4711982596938","52,615670613826",1,1,"0,50",1,1,1,1,1,1,868
7,Wall_116739,"Wall CT, Lindenufer",Berlin,Lindenufer ggü. 10 (0-24 Uhr),NaN,13597,Deutschland,"13,2077640060379","52,537351114776",1,1,"0,50",1,1,1,1,1,0,738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,Fremd_5,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,NaN,12049,Deutschland,"13,41755","52,47681",0,1,"0,00",0,0,0,0,6,0,NaN
439,Fremd_6,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,NaN,12049,Deutschland,"13,41766","52,47019",0,1,"0,00",0,0,0,0,6,0,NaN
440,Fremd_7,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,NaN,12101,Deutschland,"13,38677","52,4713",0,1,"0,00",0,0,0,0,6,0,NaN
441,Fremd_8,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,NaN,10965,Deutschland,"13,40995","52,4794",0,1,"0,00",0,0,0,0,6,0,NaN


## Exploring data

In [13]:
data.shape

(443, 19)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   LavatoryID               441 non-null    object
 1   Description              441 non-null    object
 2   City                     441 non-null    object
 3   Street                   441 non-null    object
 4   Number                   1 non-null      object
 5   PostalCode               441 non-null    object
 6   Country                  441 non-null    object
 7   Longitude                441 non-null    object
 8   Latitude                 441 non-null    object
 9   isOwnedByWall            441 non-null    object
 10  isHandicappedAccessible  441 non-null    object
 11  Price                    441 non-null    object
 12  canBePayedWithCoins      441 non-null    object
 13  canBePayedInApp          441 non-null    object
 14  canBePayedWithNFC        441 non-null    o